# Ctor

in_feats：输入特征的维度，即每个节点的特征向量的大小。  
out_feats：输出特征的维度，即图卷积操作后每个节点的特征向量的大小。  
aggregator_type：聚合器类型，指定了在邻居节点特征聚合过程中所使用的方法，例如 mean、pooling 等。  
bias：是否使用偏置项。  
norm：标准化层，用于对特征进行标准化处理。  
activation：激活函数，用于对特征进行非线性变换。  

In [21]:
import torch.nn as nn

from dgl.utils import expand_as_pair

class SAGEConv(nn.Module):
    
    def __init__(self,
                in_feats,
                out_feats,
                aggregator_type,
                bias=True,
                norm=None,
                activation=None):
        super(SAGEConv, self).__init__()
        self._in_src_feats, self._in_dst_feats = expand_as_pair(in_feats)
        self._out_feats = out_feats
        self._aggre_type = aggregator_type
        self.norm = norm
        self.activation = activation
        # 聚合类型：mean、pool、lstm、gcn
        if aggregator_type not in ['mean', 'pool', 'lstm', 'gcn']:
            raise KeyError('Aggregator type {} not supported.'.format(aggregator_type))
        if aggregator_type == 'pool':
            self.fc_pool = nn.Linear(self._in_src_feats, self._in_src_feats)
        if aggregator_type == 'lstm':
            self.lstm = nn.LSTM(self._in_src_feats, self._in_src_feats, batch_first=True)
        if aggregator_type in ['mean', 'pool', 'lstm']:
            self.fc_self = nn.Linear(self._in_dst_feats, out_feats, bias=bias)
        self.fc_neigh = nn.Linear(self._in_src_feats, out_feats, bias=bias)
        self.reset_parameters()
    
    def reset_parameters(self):
        """重新初始化可学习的参数"""
        gain = nn.init.calculate_gain('relu')
        if self._aggre_type == 'pool':
            nn.init.xavier_uniform_(self.fc_pool.weight, gain=gain)
        if self._aggre_type == 'lstm':
            self.lstm.reset_parameters()
        if self._aggre_type != 'gcn':
            nn.init.xavier_uniform_(self.fc_self.weight, gain=gain)
        nn.init.xavier_uniform_(self.fc_neigh.weight, gain=gain)